In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from datetime import datetime as dt, timezone

import os
import json
import uuid

import pandas as pd 

SERVICE_KEY = 'E:\BillTracker\gcp\service-account-key.json'
KEY = 'Jx3d2IH9lVJ6LByLVnPLOUj120B40l2FLDKYCopw'

In [2]:
# Use the application default credentials
cred = credentials.Certificate(SERVICE_KEY)
firebase_admin.initialize_app(cred)

db = firestore.client()

In [3]:
CONGRESS = '116'
CONGRESS_DATA_ROOT = f'E:/congress/data/{CONGRESS}'
BILL_DATA_ROOT = os.path.join(CONGRESS_DATA_ROOT, 'bills')
BT_DATA_ROOT = f'E:/BillTracker/data'
BT_CACHE_ROOT = f'E:/BillTracker/cache'
BT_LEGISLATORS = f'E:/BillTracker/legislators/legislators-{CONGRESS}.csv'
BT_PARTY_CACHE = os.path.join(BT_CACHE_ROOT, f'party-cache-{CONGRESS}.json')
BT_BILL_CACHE = os.path.join(BT_CACHE_ROOT, f'bill-cache-{CONGRESS}.json')
BT_BILLS = os.path.join(BT_DATA_ROOT, f'bills-{CONGRESS}.csv')

In [4]:
"""
data = load_bill_gcp(None, bill_id='hr3151-116', bill_type='hr')
db.collection('116').document(generate_bill_id_hash('hr3151')).set(data)

data = load_bill_gcp(None, bill_id='hr7120-116', bill_type='hr')
db.collection('116').document(generate_bill_id_hash('hr7120')).set(data)

data = load_bill_gcp(None, bill_id='s3548-116', bill_type='s')
db.collection('116').document(generate_bill_id_hash('s3548')).set(data)

congress_ref = db.collection(CONGRESS)

query = congress_ref.where( u'top_subject', u'in', [u'Crime and law enforcement']).order_by('status_at', direction=firestore.Query.DESCENDING).stream()

#.order_by('introduced_at', direction=firestore.Query.DESCENDING).stream()

#order_by('status_at', direction=firestore.Query.DESCENDING).stream()

for doc in query:
    print(doc.to_dict()['status_at'])
"""

"\ndata = load_bill_gcp(None, bill_id='hr3151-116', bill_type='hr')\ndb.collection('116').document(generate_bill_id_hash('hr3151')).set(data)\n\ndata = load_bill_gcp(None, bill_id='hr7120-116', bill_type='hr')\ndb.collection('116').document(generate_bill_id_hash('hr7120')).set(data)\n\ndata = load_bill_gcp(None, bill_id='s3548-116', bill_type='s')\ndb.collection('116').document(generate_bill_id_hash('s3548')).set(data)\n\ncongress_ref = db.collection(CONGRESS)\n\nquery = congress_ref.where( u'top_subject', u'in', [u'Crime and law enforcement']).order_by('status_at', direction=firestore.Query.DESCENDING).stream()\n\n#.order_by('introduced_at', direction=firestore.Query.DESCENDING).stream()\n\n#order_by('status_at', direction=firestore.Query.DESCENDING).stream()\n\nfor doc in query:\n    print(doc.to_dict()['status_at'])\n"

In [8]:
def load_bill_gcp(path, bill_id = None, bill_type = None):
    if bill_id and bill_type:
        id, congress = bill_id.split('-')
        path = 'E:\congress\data\{1}\\bills\{2}\{0}\data.json'.format(id, congress, bill_type)
        
    data = {}
    
    try: 
        with open(path, 'r') as f:
            raw_data = json.load(f)
            data['bill_id'] = raw_data['bill_id'].split('-')[0]
            data['bill_type'] = raw_data['bill_type']
            data['party'] = get_party(raw_data['sponsor']['bioguide_id'])
            data['introduced_at'] = get_datetime(raw_data['introduced_at'])
            data['status'] = raw_data['status']
            data['top_subject'] = raw_data['subjects_top_term']
            data['sponsor_id'] = raw_data['sponsor']['bioguide_id']
            data['cosponsors'] = len(raw_data['cosponsors'])
            data['status_at'] = get_datetime(raw_data['status_at'])
            data['subject'] = raw_data['subjects']
            data['summary_text'] = get_summary(raw_data['summary'])
            data['official_title'] = raw_data['official_title']
            data['popular_title'] = raw_data['popular_title']
            data['short_title'] = raw_data['short_title']
            data['display_title'] = get_display_title(raw_data['titles'])
            data['sponsor_info '] = get_sponsor_info(raw_data['sponsor'])
            data['committees'] = get_committees(raw_data['committees'])
            data['related_bills'] = get_related_bills(raw_data['related_bills'])
            data['lastmod'] = dt.astimezone(dt.now(), tz=timezone.utc)
            
    except Exception as e:
        print("Error loading: {0}", path, e)
    finally: 
        return data

load_bill_gcp('E:/congress/data/116\\bills\hr\hr1816\data.json')

{'bill_id': 'hr1816',
 'bill_type': 'hr',
 'party': 'Democrat',
 'introduced_at': datetime.datetime(2019, 3, 18, 0, 0),
 'status': 'REFERRED',
 'top_subject': 'Health',
 'sponsor_id': 'D000624',
 'cosponsors': 15,
 'status_at': datetime.datetime(2019, 3, 18, 0, 0),
 'subject': ['Child health',
  'Hazardous wastes and toxic substances',
  'Health',
  'Marketing and advertising',
  'Product safety and quality'],
 'summary_text': 'This bill does not have a summary.',
 'official_title': "To amend the Federal Food, Drug, and Cosmetic Act to require that children's cosmetics containing talc include an appropriate warning unless the cosmetics are demonstrated to be asbestos-free, and for other purposes.",
 'popular_title': None,
 'short_title': "Children's Product Warning Label Act of 2019",
 'display_title': "Children's Product Warning Label Act of 2019",
 'sponsor_info ': {'name': 'Dingell, Debbie', 'state': 'MI', 'district': '12'},
 'committees': ['House Energy and Commerce', 'House Energy

In [6]:
# Util functions 
def get_display_title(titles):
    for title in titles[::-1]:
        if title['type'] == 'display':
            return title['title']
    
    return None

def get_sponsor_info(sponsor):
    sponsor_info = {
                    'name': sponsor['name'],
                    'state': sponsor['state'],
                    'district': sponsor['district']
                   }
    
    return sponsor_info
                
def get_committees(committees):
    return [ committee['committee'] for committee in committees ]

def get_datetime(time):
    y, m, d = time.split('T')[0].split('-')
    return dt(int(y), int(m), int(d))

def get_related_bills(bills):
     return [bill['bill_id'] for bill in bills]

def generate_bill_id_hash(bill_id):
    
    class NULL_NAMESPACE:
        bytes = b''
    
    return str(uuid.uuid3(NULL_NAMESPACE, bill_id))

def get_summary(summary):
    if summary:
        return summary["text"]
    
    return "This bill does not have a summary."

In [7]:
# Party and cache access functions 
# TODO: Move legislator and party cache to function scope 

# Get legislator dataframe 
def get_legislators(path):
    return pd.read_csv(path)

legislators = get_legislators(BT_LEGISLATORS)

def get_party(id): 
    try:
        # Use legislator df to query for party type 
        return legislators[legislators['bioguide_id'] == id]['party'].item()
    except ValueError as e:
        party = check_party_cache(id)
        return party if party else get_party_req(id) 
    except Exception as e: 
        print("Unexpected Error: ", e)
        return None
    
def get_party_req(id):
    url = f'https://api.propublica.org/congress/v1/members/{id}.json'
    r = requests.get(url, headers={"X-API-Key": KEY})
    
    try:
        print("Error getting {0}. Trying alternate download.".format(id))
        r.raise_for_status()
        data = r.json()
        party = data['results'][0]['current_party']
        
        set_party_entry( id, entry={ 'party': party } )
        
        return party
    except requests.exceptions.HTTPError as e:
        print("Error getting ProPublica request.", e)
        return None
    except Exception as e:
        print("Unexpected Error: ", e)
        return None

def get_json_cache(path):
    data = {}
    
    if os.path.exists(path):
        with open(path, 'r') as f:
            data = json.load(f)
    
    return data

party_cache = get_json_cache(BT_PARTY_CACHE)

def set_party_entry(id, entry):
    party_cache[id] = entry

def dump_cache(path, cache):
    with open(path, 'w') as f:
        json.dump(cache, f, indent=4)

def check_party_cache(id):
    if id in party_cache:
        return party_cache[id]['party']
    
    return None


In [9]:
# Bill util functions
def is_bill_cache(id, lastmod, cache):
    if cache and id in cache:
        return cache[id]['lastmod']
    
    return None

def get_govinfo_lastmod(path):
    with open(path, 'r') as f:
        return f.read().strip()
    
    return None

def set_bill_cache(id, cache, entry):
    cache[id] = entry
    return cache

def get_bill_data(path):
    data = {}
    
    if os.path.exists(path):
        with open(path, 'r') as f:
            r = csv.DictReader(f, delimiter=';')
            for row in r:
                data[row['bill_id']] = row
                
        
    return data


In [11]:
len(list(get_json_cache(BT_BILL_CACHE)))

13997

In [25]:
# TODO: Test cache 
# TODO: Segment cache into seperate bill_type files

def write_bills_to_gcp(force=False):
    # Get bill cache, { bill_id: lastmod }
    bill_cache = get_json_cache(BT_BILL_CACHE)
    
    # Ref to current congress
    congress_ref = db.collection(CONGRESS)
    
    # Current bill index 
    idx = 0
    
    # Start a new write batch 
    batch = db.batch()
        
    # Iterate over bill_type root dir
    for root_path in os.scandir(BILL_DATA_ROOT):
        bill_root = os.path.basename(root_path)
        
        print("Processing directory: {0}".format(bill_root))
        
        # Go through each bill_id
        for bill_path in os.scandir(os.path.join(BILL_DATA_ROOT, root_path)): 

            data_path = os.path.join(bill_path, 'data.json')
            lastmod_path = os.path.join(bill_path, 'fdsys_billstatus-lastmod.txt')

            if os.path.exists(data_path) and os.path.exists(lastmod_path):

                lastmod = get_govinfo_lastmod(lastmod_path)
                bill_id = os.path.basename(bill_path)
                
                # Get cache result
                cache_result = is_bill_cache(bill_id, lastmod, bill_cache)
                if cache_result and not force:
                    
                    # On cache hit check if lastmods match
                    if not cache_result == lastmod:
                        print(bill_id)
                        # If they do not match, update the row 
                        bill_cache = set_bill_cache(bill_id, bill_cache, entry={ 'lastmod': lastmod })
                        
                        # Load new data for gcp 
                        bill_data = load_bill_gcp(data_path) 
                        bill_ref = congress_ref.document(generate_bill_id_hash(bill_id))
                    
                        batch.set(bill_ref, bill_data)
                 
                       
                    # If cache match, bill was not updated
                    # Do nothing 
                else:
                    print(bill_id)
                    # Update bill cache 
                    bill_cache = set_bill_cache(bill_id, bill_cache, entry={ 'lastmod': lastmod })
                    
                    bill_data = load_bill_gcp(data_path)
                    bill_ref = congress_ref.document(generate_bill_id_hash(bill_id))
                    
                    batch.set(bill_ref, bill_data)
                

                # Commit the batch at every 500th record.
                if idx % 499 == 0:
                    if idx > 0:
                        print('Committing.. ' + str(idx))
                        batch.commit()

                        # Start a new batch for the next iteration.
                        batch = db.batch()
                
                idx += 1
    
    if idx % 499 != 0:
        print('Committing.. ' + str(idx))
        batch.commit()
            
    dump_cache(BT_PARTY_CACHE, party_cache)
    dump_cache(BT_BILL_CACHE, bill_cache)

# write_bills_to_gcp()

Processing directory: hconres
Processing directory: hjres
Processing directory: hr
Committing.. 499
Committing.. 998
Committing.. 1497
Committing.. 1996
Committing.. 2495
Committing.. 2994
Committing.. 3493
Committing.. 3992
Committing.. 4491
Committing.. 4990
Committing.. 5489
Committing.. 5988
Committing.. 6487
Committing.. 6986
hr7550
hr7551
hr7552
hr7564
Committing.. 7485
Processing directory: hres
Committing.. 7984
Committing.. 8483
Processing directory: s
Committing.. 8982
Committing.. 9481
Committing.. 9980
Committing.. 10479
Committing.. 10978
Committing.. 11477
Committing.. 11976
Committing.. 12475
Committing.. 12974
Processing directory: sconres
Processing directory: sjres
Processing directory: sres
Committing.. 13473
Committing.. 13972
Committing.. 13997


In [10]:
congress_ref = db.collection(CONGRESS)

query = congress_ref.where( u'top_subject', u'in', [u'Crime and law enforcement']).order_by('status_at', direction=firestore.Query.DESCENDING).limit(20).stream()

for doc in query:
    doc_dict = doc.to_dict()
    print(doc_dict['bill_id'])
    print(doc_dict['display_title'])
    print(doc_dict['status_at'])
    print()

hr2678
No President is Above the Law Act
2020-07-23 00:00:00+00:00

s3607
Safeguarding America's First Responders Act of 2020
2020-07-20 00:00:00+00:00

hr886
Veteran Treatment Court Coordination Act of 2019
2020-07-20 00:00:00+00:00

hr7036
Antitrust Criminal Penalty Enhancement and Reform Permanent Extension Act
2020-06-25 00:00:00+00:00

s3377
Antitrust Criminal Penalty Enhancement and Reform Permanent Extension Act
2020-06-25 00:00:00+00:00

hr7120
George Floyd Justice in Policing Act of 2020
2020-06-25 00:00:00+00:00

s3985
JUSTICE Act
2020-06-24 00:00:00+00:00

sres621
A resolution designating June 15, 2020, as "World Elder Abuse Awareness Day".
2020-06-23 00:00:00+00:00

hres1013
Resolving that jurisdictions seeking to dismantle or reduce funding for the jurisdiction's police force should not unduly burden residents who seek to defend themselves.
2020-06-22 00:00:00+00:00

s4034
COVID-19 Safer Detention Act of 2020
2020-06-22 00:00:00+00:00

sres631
A resolution honoring the lif

13995
